This notebook will go over a simple rag example. We will create a collection, run a query and pass the results to an LLM, then update the collection and repeat.

In [1]:
!pip install pycolbertdb -U

In [2]:
!pip install llama-index

In [3]:
!pip install llama-index-readers-web

In [4]:
!pip install requests

In [5]:
import os
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI

from pycolbertdb.client import Colbertdb

from pycolbertdb.helpers import from_llama_index_documents

In [6]:
URL = os.getenv("COLBERTDB_URL")
API_KEY = os.getenv("COLBERTDB_API_KEY")
STORE_NAME = os.getenv("COLBERTDB_STORE_NAME")
OPEN_AI_KEY = os.getenv("COLBERTDB_OPEN_AI_KEY")
URLS = ['https://en.wikipedia.org/wiki/Onigiri']

In [7]:
client = Colbertdb(url=URL, api_key=API_KEY, store_name=STORE_NAME)
open_ai_client = OpenAI(model="gpt-4-turbo", api_key=OPEN_AI_KEY)

qa_prompt_tmpl_str = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Please write the answer in the style of {tone_name}
Query: {query_str}
Answer: \
"""

prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

In [8]:
docs = from_llama_index_documents(SimpleWebPageReader(html_to_text=True).load_data(URLS))

In [9]:
collection = client.create_collection(documents=docs, name='rice_ball_facts', options={'force_create': True})

In [10]:
result = collection.search(query="What are some popular fillings for onigiri?", k=3)

In [11]:
context = ''
for document in result.documents:
    print("Source: "+ document.metadata['source']+"\n", document.content)
    context += (document.content + "\n\n")

Source: https://en.wikipedia.org/wiki/Onigiri
 Generally, onigiri made with pre-
seasoned rice is not filled with ingredients. Plain (salt only) onigiri is
called _shio-musubi_.

Typical fillings are listed below:

  * Dressed dishes: tuna with mayonnaise (シーチキン), shrimp with mayonnaise, [negitoro](/wiki/Negitoro "Negitoro") (ネギトロ), etc.
  * Dried fish: roasted and crumbled [mackerel](/wiki/Mackerel_as_food "Mackerel as food") (鯖), [Japanese horse mackerel](/wiki/Japanese_horse_mackerel "Japanese horse mackerel") (鰺), etc.
  * [Kakuni](/wiki/Kakuni "Kakuni"): [dongpo pork](/wiki/Dongpo_pork "Dongpo pork")
  * [Dried food](/wiki/Drying_\(food\) "Drying \(food\)"): [okaka](/wiki/Katsuobushi "Katsuobushi"), etc.
Source: https://en.wikipedia.org/wiki/Onigiri
 Because it is easily portable and eaten by hand, onigiri has been
used as portable food or [bento](/wiki/Bento "Bento") from ancient times to
the present day. Originally, it was used as a way to use and store left-over
rice, but it la

In [12]:
prompt = prompt_tmpl.format(context_str=context, tone_name="shakespeare", query_str="What are some typical onigiri fillings")
open_ai_client.complete(prompt)

CompletionResponse(text="Lo, hear me speak of onigiri's heart, where flavors rich and varied start:\nDressed dishes doth their charm impart with tuna mixed with creamy art,\nAnd shrimp that swims in mayonnaise, negitoro in its zesty ways.\n\nDried fish, too, a tale to tell, of mackerel's smoke and horse mackerel's swell,\nEach crumbled o'er the rice so well, a story in each bite doth dwell.\n\nKakuni, with its pork so sweet, in dongpo's style, a savory treat,\nAnd dried delights, not to forget, okaka's dance of flavors met.\n\nThese fillings all, in rice ensconced, by hands adept are deftly launched\nInto the mouths of those who yearn for onigiri at every turn.", additional_kwargs={}, raw={'id': 'chatcmpl-9WRnhWVBPIpCI5ygXpFWXLlXPLA5b', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Lo, hear me speak of onigiri's heart, where flavors rich and varied start:\nDressed dishes doth their charm impart with tuna mixed with creamy art,\n

In [13]:
new_docs = SimpleWebPageReader(html_to_text=True).load_data(["https://en.wikipedia.org/wiki/Kewpie_(mayonnaise)"])
new_formatted = [{"content": doc.text, "metadata": {"source": doc.id_}} for doc in new_docs[0:2]]

In [14]:
collection = collection.add_documents(documents=new_formatted)

In [15]:
new_result = collection.search(query="When was kewpie mayo founded?", k=3)
new_context = ''
for document in new_result.documents:
    print("Source: "+ document.metadata['source']+"\n", document.content)
    new_context += (document.content + "\n\n")

Source: https://en.wikipedia.org/wiki/Kewpie_(mayonnaise)
 Ltd. _食品工業株式会社_ was founded in [Nakano,
Tokyo](/wiki/Nakano,_Tokyo "Nakano, Tokyo") in 1919 by Toichiro Nakashima.[2]
He had previously worked in the United States for three years as an intern for
the [Ministry of Agriculture and
Commerce](/wiki/Ministry_of_Agriculture_and_Commerce "Ministry of Agriculture
and Commerce"), where he first encountered mayonnaise and the [Kewpie
doll](/wiki/Kewpie_doll "Kewpie doll") that became his emblem.[3] Originally,
his business distributed canned salmon.[2] Mayonnaise production began in
1925,[4] and it has been the best selling brand of mayonnaise in Japan ever
since.[5] The company is now called the Kewpie Corp. The company stopped
production of mayonnaise during World War II because of supply shortages, and
resumed production in 1948.
Source: https://en.wikipedia.org/wiki/Kewpie_(mayonnaise)
 Kewpie is the best selling mayonnaise in Japan,[1] and is also sold
in other countries.

##
Histo

In [16]:
prompt = prompt_tmpl.format(context_str=new_context, tone_name="bruce springsteen", query_str="When and where was kewpie mayo founded")
open_ai_client.complete(prompt)

CompletionResponse(text="Well, let me take you back to the year 1919, in the heart of Nakano, Tokyo. That's where and when the story of Kewpie mayo began, all thanks to a fella named Toichiro Nakashima. He started it all, bringing that smooth, creamy taste right into the heart of Japan.", additional_kwargs={}, raw={'id': 'chatcmpl-9WRnrTIFm3axzXKRQrIdCoINa78Iu', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Well, let me take you back to the year 1919, in the heart of Nakano, Tokyo. That's where and when the story of Kewpie mayo began, all thanks to a fella named Toichiro Nakashima. He started it all, bringing that smooth, creamy taste right into the heart of Japan.", role='assistant', function_call=None, tool_calls=None))], 'created': 1717520399, 'model': 'gpt-4-turbo-2024-04-09', 'object': 'chat.completion', 'system_fingerprint': 'fp_31a6c0accf', 'usage': CompletionUsage(completion_tokens=69, prompt_tokens=710, total_tokens=779